# Example of generating QAs for a 10K
Source: https://investors.nike.com/investors/news-events-and-reports/

### Before running the code

You will need to have the following packages installed:
```
pip3 install langchain pandas pypdf
```

Also, make sure you have a .env file with your OpenAI API key in the root directory of this project.
```
OPENAI_API_KEY=YOUR_API_KEY
```

### Load packages

In [1]:
%reload_ext autoreload
%autoreload 2

import sys

sys.path.append(".")
sys.path.append("..")
sys.path.append("../..")

In [2]:
from dotenv import load_dotenv
import os
import pandas as pd
from uniflow.client import Client
from uniflow.config import OpenAIConfig
from uniflow.model.config import OpenAIModelConfig
from langchain.document_loaders import PyPDFLoader
from uniflow.schema import Context, GuidedPrompt

load_dotenv()


/Users/joseortiz/anaconda3/envs/uniflow/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

### Prepare the input data

In [3]:
pdf_file = "nike-10k-2023.pdf"

##### Set current directory and input data directory.

In [4]:
dir_cur = os.getcwd()
input_file = os.path.join(f"{dir_cur}/data/raw_input/", pdf_file)

##### Load and split the pdf

In [5]:
loader = PyPDFLoader(input_file)
pages = loader.load_and_split()
page_contents = [page.page_content for page in pages]

In [6]:
guided_prompt = GuidedPrompt(
    examples=[
        Context(
            context="In 1948, Claude E. Shannon published A Mathematical Theory of\nCommunication (Shannon, 1948) establishing the theory of\ninformation. In his article, Shannon introduced the concept of\ninformation entropy for the first time. We will begin our journey here.",
            question="Who published A Mathematical Theory of Communication in 1948?",
            answer="Claude E. Shannon.",
        ),
])

data = [ Context(context=p[:500]) for p in page_contents[6:16] if len(p) > 200]

In [7]:
data

[Context(context='We also offer interactive consumer services and experiences as well as digital products through our digital platforms, including \nfitness and activity apps; sport, fitness and wellness content; and digital services and features in retail stores that enhance the \nconsumer experience.\nSALES AND MARKETING\nWe experience moderate fluctuations in aggregate sales volume during the year. Historically, revenues in the first and fourth \nfiscal quarters have slightly exceeded those in the second and third '),
 Context(context='INTERNATIONAL MARKETS\nFor fiscal 2023, non-U.S. NIKE Brand and Converse sales accounted for approximately 57% of total revenues, compared to 60% \nand 61% for fiscal 2022 and fiscal 2021, respectively. We sell our products to retail accounts through our own NIKE Direct \noperations and through a mix of independent distributors, licensees and sales representatives around the world. W e sell to \nthousands of retail accounts and ship products from 67 d

### Run the model

In [8]:
config = OpenAIConfig(
    guided_prompt_template=guided_prompt,
    model_config=OpenAIModelConfig(response_format={"type": "json_object"}),
)
client = Client(config)

In [9]:
output = client.run(data)

100%|██████████| 10/10 [00:19<00:00,  1.98s/it]


In [10]:
output

[{'output': [{'response': [{'context': 'We also offer interactive consumer services and experiences as well as digital products through our digital platforms, including \nfitness and activity apps; sport, fitness and wellness content; and digital services and features in retail stores that enhance the \nconsumer experience.\nSALES AND MARKETING\nWe experience moderate fluctuations in aggregate sales volume during the year. Historically, revenues in the first and fourth \nfiscal quarters have slightly exceeded those in the second and third ',
      'question': 'What kind of digital products do the mentioned digital platforms offer?',
      'answer': 'The digital platforms offer fitness and activity apps, sport, fitness and wellness content, and digital services and features in retail stores that enhance the consumer experience.'}],
    'error': 'No errors.'}],
  'root': <uniflow.node.node.Node at 0x122e68e20>},
 {'output': [{'response': [{'context': 'INTERNATIONAL MARKETS\nFor fiscal 20

### Process the output

In [11]:
# Extracting context, question, and answer into a DataFrame
contexts = []
questions = []
answers = []

for item in output:
    for i in item.get('output', []):
        for response in i.get('response', []):
            if any(key not in response for key in ['context', 'question', 'answer']):
                print("Missing context, question or answer in response:", response)
                continue
            contexts.append(response['context'])
            questions.append(response['question'])
            answers.append(response['answer'])

# Set display options
pd.set_option('display.max_colwidth', None)  # or use a specific width like 50
pd.set_option('display.width', 1000)

df = pd.DataFrame({
    'Context': contexts,
    'Question': questions,
    'Answer': answers
})

df

,Context,Question,Answer
0,"We also offer interactive consumer services and experiences as well as digital products through our digital platforms, including \nfitness and activity apps; sport, fitness and wellness content; and digital services and features in retail stores that enhance the \nconsumer experience.\nSALES AND MARKETING\nWe experience moderate fluctuations in aggregate sales volume during the year. Historically, revenues in the first and fourth \nfiscal quarters have slightly exceeded those in the second and third",What kind of digital products do the mentioned digital platforms offer?,"The digital platforms offer fitness and activity apps, sport, fitness and wellness content, and digital services and features in retail stores that enhance the consumer experience."
1,"INTERNATIONAL MARKETS\nFor fiscal 2023, non-U.S. NIKE Brand and Converse sales accounted for approximately 57% of total revenues, compared to 60% \nand 61% for fiscal 2022 and fiscal 2021, respectively. We sell our products to retail accounts through our own NIKE Direct \noperations and through a mix of independent distributors, licensees and sales representatives around the world. W e sell to \nthousands of retail accounts and ship products from 67 distribution centers outside of the United States.",How much did non-U.S. Brand and Converse sales account for in fiscal 2023?,Approximately 57% of total revenues.
2,"footwear production. For fiscal 2023, factories in Vietnam, Indonesia and China manufactured approximately 50%, 27% and 18% \nof total NIKE Brand footwear, respectively. For fiscal 2023, four footwear contract manufacturers each accounted for greater than \n10% of footwear production and in the aggregate accounted for approximately 58% of NIKE Brand footwear production.\nAs of May 31, 2023, our contract manufacturers operated 291 finished goods apparel factories located in 31 countries. For fiscal","Which countries manufactured approximately 50%, 27% and 18% of the total NIKE Brand footwear in 2023?","Vietnam, Indonesia, and China."
3,"NIKE's contract manufacturers buy raw materials for the manufacturing of our footwear, apparel and equipment products. Most raw materials are available and purchased by those contract manufacturers in the countries where manufact",Where are most raw materials purchased by NIKE's contract manufacturers?,Most raw materials are purchased by NIKE's contract manufacturers in the countries where manufacturing takes place.
4,"We monitor protectionist trends and developments throughout the world that may materially impact our industry, and we engage \nin administrative and judicial processes to mitigate trade restrictions. W e are actively monitoring actions that may result in \nadditional anti-dumping measures and could affect our industry. We are also monitoring for and advocating against other \nimpediments that may limit or delay customs clearance for imports of footwear , apparel and equipment. NIKE also advocates f",What does NIKE monitor and advocate against that may affect customs clearance for imports?,"NIKE monitors and advocates against impediments that may limit or delay customs clearance for imports of footwear, apparel, and equipment."
5,"Our international operations are also subject to compliance with the U.S . Foreign Corrupt Practices Act (the ""FCPA""), and other \nanti-bribery laws applicable to our operations. We source a significant portion of our products from, and have important consumer \nmarkets, outside of the United States. We have an ethics and compliance program to address compliance with the FCPA and \nsimilar laws by us, our employees, agents, suppliers and other partners. Refer to Item 1A. Risk Factors for additiona",What laws are applicable to the international operations?,The U.S. Foreign Corrupt Practices Act (the 'FCPA') and other anti-bribery laws applicable to the operations.
6,"devices, and related software applications. These patents expire at various times.\nWe believe our succ

In [12]:
output_df = df[['Question', 'Answer']]

output_dir = 'data/output'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

output_df.to_csv(f"{output_dir}/Nike_10k_QApairs.csv", index=False)